# Load dataset 7

In [1]:
import scanpy as sc
from pathlib import Path
import tarfile

In [2]:
base_dir = Path(r"C:\Users\clark\OneDrive\Documents\GitHub\CNA_tool\data\data7\GSE195467_RAW")

for tar_file in base_dir.glob("*.tar.gz"):
    extract_dir = base_dir / tar_file.stem  # e.g., GSM5837936_sample1_t1
    extract_dir.mkdir(exist_ok=True)
    print(f"Extracting: {tar_file.name} → {extract_dir.name}")
    with tarfile.open(tar_file, mode="r:gz") as tar:
        tar.extractall(path=extract_dir)


Extracting: GSM5837936_sample1_t1.tar.gz → GSM5837936_sample1_t1.tar
Extracting: GSM5837937_sample1_t2.tar.gz → GSM5837937_sample1_t2.tar
Extracting: GSM5837938_sample1_t3.tar.gz → GSM5837938_sample1_t3.tar
Extracting: GSM5837939_sample1_t4.tar.gz → GSM5837939_sample1_t4.tar
Extracting: GSM5837940_sample2_t1.tar.gz → GSM5837940_sample2_t1.tar
Extracting: GSM5837941_sample2_t2.tar.gz → GSM5837941_sample2_t2.tar
Extracting: GSM5837942_sample2_t3.tar.gz → GSM5837942_sample2_t3.tar
Extracting: GSM5837943_sample3_t1.tar.gz → GSM5837943_sample3_t1.tar
Extracting: GSM5837944_sample3_t2.tar.gz → GSM5837944_sample3_t2.tar


In [ ]:

adatas = []

for outer in base_dir.iterdir():
    if outer.is_dir():
        # Look for exactly one subdirectory inside
        subdirs = [d for d in outer.iterdir() if d.is_dir()]
        if len(subdirs) == 1 and (subdirs[0] / "matrix.mtx.gz").exists():
            print(f"Loading: {subdirs[0]}")
            adata = sc.read_10x_mtx(subdirs[0], var_names="gene_symbols")
            adata.obs["sample"] = outer.name
            adatas.append(adata)
        else:
            print(f"Skipping: {outer} (no valid matrix.mtx.gz found)")

# Merge and save
if adatas:
    adata_all = adatas[0].concatenate(
        *adatas[1:], 
        batch_key="sample_id",
        batch_categories=[a.obs["sample"][0] for a in adatas]
    )
    out_path = base_dir / "GSE195467_merged.h5ad"
    adata_all.write(out_path)
    print(f"Merged .h5ad saved to: {out_path}")
else:
    print("Still no valid samples found.")


# load processed data

In [3]:
num4_adata_path = "../data/num4_adata.h5ad"
adata4 = sc.read_h5ad(num4_adata_path)
print(f"Loaded: {num4_adata_path}")
adata4.obs

Loaded: ../data/num4_adata.h5ad


,sample
AAACCCAAGAACTGAT-1_PGP1_WT2,PGP1_WT2
AAACCCAAGACGCCAA-1_PGP1_WT2,PGP1_WT2
AAACCCAAGATGTAGT-1_PGP1_WT2,PGP1_WT2
AAACCCAAGCCTATCA-1_PGP1_WT2,PGP1_WT2
AAACCCAAGGCACTAG-1_PGP1_WT2,PGP1_WT2
...,...
TTTGTTGGTGCCTGCA-1_WTC11_Het1,WTC11_Het1
TTTGTTGTCAACGCTA-1_WTC11_Het1,WTC11_Het1
TTTGTTGTCCCGGTAG-1_WTC11_Het1,WTC11_Het1
TTTGTTGTCGAAGTGG-1_WTC11_Het1,WTC11_Het1


In [2]:
num6_adata_path = "../data/num6_adata.h5ad"
adata6 = sc.read_h5ad(num6_adata_path)
print(f"Loaded: {num6_adata_path}")

Loaded: ../data/num6_adata.h5ad


In [ ]:
num3_adata_path = "../data/num3_adata.h5ad"
adata3 = sc.read_h5ad(num3_adata_path)
print(f"Loaded: {num6_adata_path}")

Loaded: ../data/num6_adata.h5ad


/Users/apple/opt/miniconda3/envs/scFates/lib/python3.11/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [30]:
num7_adata_path = "../data/num7_adata.h5ad"
adata7 = sc.read_h5ad(num7_adata_path)
print(f"Loaded: {num7_adata_path}")

Loaded: ../data/num7_adata.h5ad


In [4]:
adata = sc.read_h5ad("PBMC_simulated_cnas_041025.h5ad")

In [19]:
adata.var

,gene_ids,feature_types,genome,mt,ribo,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,chromosome,start,end,strand
original_index,,,,,,,,,,,,,,
AL627309.1,ENSG00000238009,Gene Expression,GRCh38,False,False,60,0.005183,99.490186,61.0,60,NaN,NaN,NaN,NaN
AL627309.3,ENSG00000239945,Gene Expression,GRCh38,False,False,4,0.000340,99.966012,4.0,4,1,89551.0,91105.0,-1.0
AL669831.5,ENSG00000237491,Gene Expression,GRCh38,False,False,679,0.062367,94.230606,734.0,673,1,778739.0,810066.0,1.0
FAM87B,ENSG00000177757,Gene Expression,GRCh38,False,False,13,0.001190,99.889540,14.0,13,1,817363.0,819842.0,1.0
LINC00115,ENSG00000225880,Gene Expression,GRCh38,False,False,350,0.031269,97.026085,368.0,340,1,586945.0,827989.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC011043.1,ENSG00000276256,Gene Expression,GRCh38,False,False,77,0.006882,99.345739,81.0,75,GL000195.1,42939.0,49164.0,-1.0
AL592183.1,ENSG00000273748,Gene Expression,GRCh38,False,False,32,0.002719,99.728099,32.0,31,GL000219.1,54224.0,83311.0,-1.0
AC007325.4,ENSG00000278817,Gene Expression,GRCh38,False,False,239,0.020902,97.969241,246.0,235,KI270734.1,131494.0,137392.0,1.0


# Task 3 

In [3]:
import sys
sys.path.append('../src')

#from preprocessing import annotation_preprocess
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cna_tool
from cna_tool import CNAInferer  # assuming this contains your working class
from cna_tool.utils import select_control_mask
from cna_tool.infer import infer_cnas_from_scrna
from cna_tool.tl import run_cna_evaluation
from cna_tool.cna_inference import test_pipeline_on_slice

# functions

In [24]:
from mygene import MyGeneInfo

def fetch_gene_coordinates_via_mygene(adata, species='human'):
    mg = MyGeneInfo()
    gene_names = adata.var['gene_name'].astype(str).unique().tolist()

    print(f"Querying {len(gene_names)} genes from mygene.info...")
    results = mg.querymany(gene_names, scopes='symbol', fields='genomic_pos', species=species, as_dataframe=True)
    results = results[~results.index.duplicated(keep='first')]  # remove duplicate matches

    def get_coord(gene, field):
        try:
            val = results.loc[gene, 'genomic_pos']
            if isinstance(val, dict):
                return val.get(field)
            elif isinstance(val, list) and isinstance(val[0], dict):
                return val[0].get(field)
            else:
                return None
        except Exception:
            return None

    adata.var['chromosome'] = adata.var['gene_name'].map(lambda g: get_coord(g, 'chr'))
    adata.var['start'] = adata.var['gene_name'].map(lambda g: get_coord(g, 'start'))
    adata.var['end'] = adata.var['gene_name'].map(lambda g: get_coord(g, 'end'))

    return adata


In [25]:
def load_gtf_as_dataframe(gtf_file):
    rows = []
    with open(gtf_file, 'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            fields = line.strip().split('\t')
            if fields[2] != 'gene':
                continue
            chrom, source, feature, start, end, score, strand, frame, attr = fields
            info = {k: v.strip('"') for k, v in 
                    [field.strip().split(' ')[:2] for field in attr.strip(';').split(';') if field]}
            rows.append({
                'gene_name': info.get('gene_name'),
                'gene_id': info.get('gene_id'),
                'chromosome': chrom.replace('chr', ''),
                'start': int(start),
                'end': int(end)
            })
    return pd.DataFrame(rows)

# dataset 6

In [26]:
num6_adata_path = "../data/num6_adata.h5ad"
adata6 = sc.read_h5ad(num6_adata_path)
print(f"Loaded: {num6_adata_path}")

adata6.var['gene_name'] = adata6.var.index
adata6.var


Loaded: ../data/num6_adata.h5ad


,gene_name
A1BG,A1BG
A1BG-AS1,A1BG-AS1
A1CF,A1CF
A2M,A2M
A2M-AS1,A2M-AS1
...,...
ZZZ3,ZZZ3
bP-21264C1.2,bP-21264C1.2
bP-2171C21.3,bP-2171C21.3
bP-2189O9.3,bP-2189O9.3


In [27]:
adata6 = fetch_gene_coordinates_via_mygene(adata6)
print(adata6.var[['gene_name', 'chromosome', 'start', 'end']].head(10))
print("Missing values:", adata6.var[['chromosome', 'start', 'end']].isna().sum())


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


Querying 45068 genes from mygene.info...


1323 input query terms found dup hits:	[('A2ML1-AS1', 2), ('A2ML1-AS2', 2), ('AADACL2-AS1', 3), ('ABHD15-AS1', 2), ('ACBD3-AS1', 2), ('ADAM
22411 input query terms found no hit:	['AAED1', 'AARS', 'AATK-AS1', 'AB015752.1', 'AB015752.3', 'AB019440.50', 'ABC11-4932300O16.1', 'ABC1


           gene_name chromosome  start  end
A1BG            A1BG       None    NaN  NaN
A1BG-AS1    A1BG-AS1       None    NaN  NaN
A1CF            A1CF       None    NaN  NaN
A2M              A2M       None    NaN  NaN
A2M-AS1      A2M-AS1       None    NaN  NaN
A2ML1          A2ML1       None    NaN  NaN
A2ML1-AS1  A2ML1-AS1       None    NaN  NaN
A2ML1-AS2  A2ML1-AS2       None    NaN  NaN
A3GALT2      A3GALT2       None    NaN  NaN
A4GALT        A4GALT       None    NaN  NaN
Missing values: chromosome    43538
start         43538
end           43538
dtype: int64


use gtf

In [28]:
gtf_df1 = load_gtf_as_dataframe("gencode.v47.annotation.gtf")

# this is more comprehensive
gtf_df = load_gtf_as_dataframe("gencode.v47.chr_patch_hapl_scaff.annotation.gtf")


In [29]:
# Step 1: Copy and reset gene_name
merged = adata6.var.copy()
merged['gene_name'] = merged.index.astype(str)

# Step 2: Merge with GTF
merged = merged.merge(gtf_df, on='gene_name', how='left')

# Step 3: Drop duplicates (if merge resulted in more rows)
merged = merged.drop_duplicates(subset='gene_name', keep='first')

# Step 4: Align merged index with adata6.var
merged = merged.set_index('gene_name')
merged = merged.loc[adata6.var.index]  # ensure same order and rows

# Step 5: Assign to adata6.var
adata6.var['chromosome'] = merged['chromosome_y'].fillna(merged['chromosome_x'])
adata6.var['start'] = merged['start_y'].fillna(merged['start_x'])
adata6.var['end'] = merged['end_y'].fillna(merged['end_x'])



In [30]:
print("Final gene coordinate coverage:")
print(adata6.var[['chromosome', 'start', 'end']].notna().sum())


Final gene coordinate coverage:
chromosome    22548
start         22548
end           22548
dtype: int64


In [33]:
nan_count = adata6.var['chromosome'].isna().sum()
print(f"Number of NaN values in adata6.var['chromosome']: {nan_count}")

Number of NaN values in adata6.var['chromosome']: 22520


In [34]:
adata6.var

,gene_name,chromosome,start,end
A1BG,A1BG,19,58345178.0,58353492.0
A1BG-AS1,A1BG-AS1,19,58347718.0,58355455.0
A1CF,A1CF,10,50799409.0,50885675.0
A2M,A2M,12,9067664.0,9116229.0
A2M-AS1,A2M-AS1,12,9065163.0,9068689.0
...,...,...,...,...
ZZZ3,ZZZ3,1,77562416.0,77683419.0
bP-21264C1.2,bP-21264C1.2,None,NaN,NaN
bP-2171C21.3,bP-2171C21.3,None,NaN,NaN
bP-2189O9.3,bP-2189O9.3,None,NaN,NaN


# dataset 4

In [35]:
num4_adata_path = "../data/num4_adata.h5ad"
adata4 = sc.read_h5ad(num4_adata_path)
print(f"Loaded: {num4_adata_path}")

adata4.var['gene_name'] = adata4.var.index
adata4.var


Loaded: ../data/num4_adata.h5ad


,gene_name
GRCh38_MIR1302-2HG,GRCh38_MIR1302-2HG
GRCh38_FAM138A,GRCh38_FAM138A
GRCh38_OR4F5,GRCh38_OR4F5
GRCh38_AL627309.1,GRCh38_AL627309.1
GRCh38_AL627309.3,GRCh38_AL627309.3
...,...
mm10___AC124606.1,mm10___AC124606.1
mm10___AC133095.2,mm10___AC133095.2
mm10___AC133095.1,mm10___AC133095.1
mm10___AC234645.1,mm10___AC234645.1


In [ ]:
adata4 = fetch_gene_coordinates_via_mygene(adata4)
print(adata4.var[['gene_name', 'chromosome', 'start', 'end']].head(10))
print("Missing values:", adata4.var[['chromosome', 'start', 'end']].isna().sum())


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


Querying 68886 genes from mygene.info...


In [38]:
gtf_df = load_gtf_as_dataframe("gencode.v47.chr_patch_hapl_scaff.annotation.gtf")

In [36]:
# Step 1: Copy and reset gene_name
merged = adata4.var.copy()
merged['gene_name'] = merged.index.astype(str)

# Step 2: Merge with GTF
merged = merged.merge(gtf_df, on='gene_name', how='left')

# Step 3: Drop duplicates (if merge resulted in more rows)
merged = merged.drop_duplicates(subset='gene_name', keep='first')

# Step 4: Align merged index with adata4.var
merged = merged.set_index('gene_name')
merged = merged.loc[adata4.var.index]  # ensure same order and rows

# Step 5: Assign to adata4.var
adata4.var['chromosome'] = merged['chromosome_y'].fillna(merged['chromosome_x'])
adata4.var['start'] = merged['start_y'].fillna(merged['start_x'])
adata4.var['end'] = merged['end_y'].fillna(merged['end_x'])



KeyError: 'chromosome_y'

# infer CNA

In [10]:
adata = infer_cnas_from_scrna(adata)

/Users/apple/Documents/GitHub/CNA_tool/tests/../src/cna_tool/infer.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  largest_group = adata.obs.groupby('cell_type').size().idxmax()


[info] No control mask provided. Using largest cluster ('CD4 T cell') as control.


/Users/apple/Documents/GitHub/CNA_tool/tests/../src/cna_tool/utils.py:155: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for chrom, sub in var.groupby('chromosome'):


In [11]:
adata6.obs

,sample
AAACCTGAGAATTGTG-1_GSM3814885_day0,GSM3814885_day0
AAACCTGAGACAATAC-1_GSM3814885_day0,GSM3814885_day0
AAACCTGAGACTACAA-1_GSM3814885_day0,GSM3814885_day0
AAACCTGAGACTTTCG-1_GSM3814885_day0,GSM3814885_day0
AAACCTGAGGCGTACA-1_GSM3814885_day0,GSM3814885_day0
...,...
TTTGTCACACATCCAA-1_GSM3814900_huvec,GSM3814900_huvec
TTTGTCACATGCAACT-1_GSM3814900_huvec,GSM3814900_huvec
TTTGTCAGTTGATTGC-1_GSM3814900_huvec,GSM3814900_huvec
TTTGTCATCTAACTCT-1_GSM3814900_huvec,GSM3814900_huvec
